El objetivo de este este código es implementar un GBM dada una partición, y dados los datos.

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/repo_tesis/entorno_tesis_Molina"
!source bin/activate

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/repo_tesis/entorno_tesis_Molina


In [17]:
# !pip install comet_ml
from comet_ml import Experiment
import numpy as np
from sklearn.model_selection import train_test_split
import sys
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import joblib
# import seaborn as sns
import time
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt

sys.path.append('/content/drive/My Drive/repo_tesis/lib')
import classes as cl

In [15]:
# Importo los datos
features = np.array(joblib.load("/content/drive/My Drive/repo_tesis/data/FEATURES_W200_I50_PCA95.joblib"))
label = np.array(joblib.load("/content/drive/My Drive/repo_tesis/data/label_W200_I50.joblib"))

%cd "/content/drive/My Drive/repo_tesis/archivos_generados_codigos"
# Importo particiones
indices_train = np.array(joblib.load('indices_train_W200_I50.pkl'))
indices_val = np.array(joblib.load('indices_val_W200_I50.pkl'))
indices_test = np.array(joblib.load('indices_test_W200_I50.pkl'))



/content/drive/My Drive/repo_tesis/archivos_generados_codigos


In [18]:
start_time = time.time()

# implemento gbm
clf = LGBMClassifier()
clf.fit(features[indices_train, :], label[indices_train, 1], eval_set=[(features[indices_val, :], label[indices_val, 1])], eval_metric='logloss')

end_time = time.time()
training_time = end_time - start_time
# Obtener las iteraciones del mejor modelo
bst = clf.best_iteration_

nombre = 'ronda1_exp_gbm_sep_rep_pca95_n1'

joblib.dump(clf, nombre + '.pkl')

y_pred = clf.predict(features[indices_test, :])

accuracy = accuracy_score(label[indices_test, 1], y_pred)
precision = precision_score(label[indices_test, 1], y_pred, average='weighted')
recall = recall_score(label[indices_test, 1], y_pred, average='weighted')
conf_matrix = confusion_matrix(label[indices_test, 1], y_pred)
plt.savefig("confusion_matrix.png")

API_KEY = 'ehXeElNypcj7Knar5zTmyjwSO'

exp = Experiment(api_key=API_KEY,
                project_name='tesis-experimentos', # Nombre del proyecto donde se registran los experimentos
                auto_param_logging=False)
exp.set_name(nombre) # Nombre de este experimento
exp.add_tags(['exp1', 'gbm', 'sep_rep', 'pca95']) # Tags

exp.log_metric("accuracy", accuracy)
exp.log_metric("precision", precision)
exp.log_metric("recall", recall)
exp.log_metric("training_time", training_time)
exp.log_confusion_matrix(label[indices_test, 1], y_pred)
exp.log_parameter("ind_train", indices_train)
exp.log_parameter("ind_val", indices_val)
exp.log_parameter("ind_test", indices_test)
exp.log_text("Experimento separando los datos por repeticiones y usando GBM. En cuanto a las ventanas: W=200, I=50.")   # Comentario del experimento
exp.log_model(name=nombre, file_or_folder=nombre + '.pkl')
exp.end()

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009725 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11220
[LightGBM] [Info] Number of data points in the train set: 43046, number of used features: 44
[LightGBM] [Info] Start training from score -2.564113
[LightGBM] [Info] Start training from score -2.604759
[LightGBM] [Info] Start training from score -2.617410
[LightGBM] [Info] Start training from score -2.593199
[LightGBM] [Info] Start training from score -2.580542
[LightGBM] [Info] Start training from score -2.552712
[LightGBM] [Info] Start training from score -2.554803
[LightGBM] [Info] Start training from score -2.555999
[LightGBM] [Info] Start training from score -2.558697
[LightGBM] [Info] Start training from score -2.601622
[LightGBM] [Info] Start training from score -2.248242
[LightGBM] [Info] Start training from score -2.55809

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/636f2a0bd87c455aa0645e69277056c4

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/636f2a0bd87c455aa0645e69277056c4
COMET INFO:   Metrics:
COMET INFO:     accuracy      : 0.5350449775112444
COMET INFO:     precision     : 0.5628329296997877
COMET INFO:     recall        : 0.5350449775112444
COMET INFO:     training_time : 28.55322813987732
COMET INFO:   Others:
COMET INFO

<Figure size 640x480 with 0 Axes>